# Training CNN Model on EuroSat Dataset


This notebook loads the EuroSat dataset, preprocesses images, splits data into training and testing sets, 
trains a CNN model, evaluates its performance, and saves the trained model.


In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
from sklearn.model_selection import train_test_split


In [2]:

# Define parameters
batch_size = 32
num_epochs = 10
learning_rate = 0.001
model_path = "eurosat_cnn.pth"
train_ratio = 0.8  # 80% training, 20% testing

# Define transforms
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:

# Load the dataset from the 'eurosat' folder
data_dir = "EuroSat"
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset into train and test sets
train_size = int(train_ratio * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Create DataLoader for training and testing
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [4]:

# Define a simple CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)  # Multi-class classification
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:

# Get the number of classes dynamically
num_classes = len(full_dataset.classes)

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [6]:

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 1.0121413961163273
Epoch 2/10, Loss: 0.5944973308731009
Epoch 3/10, Loss: 0.4804928371199855
Epoch 4/10, Loss: 0.4143255316990393
Epoch 5/10, Loss: 0.3458003600879952
Epoch 6/10, Loss: 0.2978402491631331
Epoch 7/10, Loss: 0.25415200796116283
Epoch 8/10, Loss: 0.21348034047694117
Epoch 9/10, Loss: 0.18010920121851895
Epoch 10/10, Loss: 0.14663859374269292


In [7]:

# Save trained model
torch.save(model.state_dict(), model_path)


In [8]:

# Validate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 88.06%
